In [1]:
import camelot
from PyPDF2 import PdfFileWriter, PdfFileReader
import os
import shutil
import pickle
import pandas as pd
import xml.etree.ElementTree as ET


In [2]:
def get_labels(xml_path):

    tree = ET.parse(xml_path)
    document = tree.getroot()
    flag = 0
    Table_Num = 1
    content = {} ## content[table_number] = list of tokens in that table
    for page in document:
        if page.tag == "PAGE":
            #print(page.tag,"asdfasdf")
            page_attrib = page.attrib
            p_num = page_attrib["number"]
            #print(p_num)
            for text in page:
                for token in text:
                    if token.text == "#THIS#IS#START#OF#TABLE#":
                        flag = 1
                    elif token.text == "#THIS#IS#END#OF#TABLE#":
                        flag = 0
                        Table_Num+=1
                    if flag:
                        if str(Table_Num)+"_"+str(p_num) not in content:
                            content[str(Table_Num)+"_"+str(p_num)] = [token.text]
                        else:
                            content[str(Table_Num)+"_"+str(p_num)].append(token.text)
                    
                
    return content

In [4]:
c = get_labels(os.getcwd()+"/xmls/"+"0704.1028.xml")

In [5]:
c

{'1_5': ['#THIS#IS#START#OF#TABLE#',
  'Mean',
  'zero-one',
  'error',
  'Mean',
  'absolute',
  'error',
  'Dataset',
  'NNRank',
  'NNClass',
  'NNRank',
  'NNClass',
  'Stocks',
  '12.68±1.8%',
  '16.97±',
  '2.3%',
  '0.127±0.01',
  '0.173±0.02',
  'Pyrimidines',
  '37.71±8.1%',
  '41.87±7.9%',
  '0.450±0.09',
  '0.508±0.11',
  'Auto',
  'MPG',
  '27.13±2.0%',
  '28.82±2.7%',
  '0.281±0.02',
  '0.307±0.03',
  'Machine',
  '17.03±4.2%',
  '17.80±4.4%',
  '0.186±0.04',
  '0.192±0.06',
  'Abalone',
  '21.39±0.3%',
  '21.74±',
  '0.4%',
  '0.226±0.01',
  '0.232±0.01',
  'Triazines',
  '52.55±5.0%',
  '52.84±5.9%',
  '0.730±0.06',
  '0.790±0.09',
  'Boston',
  '26.38±3.0%',
  '26.62±2.7%',
  '0.295±0.03',
  '0.297±0.03',
  'Diabetes',
  '44.90±12.5%',
  '43.84±10.0%',
  '0.546±0.15',
  '0.592±0.09'],
 '2_5': ['#THIS#IS#START#OF#TABLE#',
  'Data',
  'NNRank',
  'SVM',
  'GP-MAP',
  'GP-EP',
  'Triazines',
  '52.55±5.0%',
  '54.19±1.5%',
  '52.91±2.2%',
  '52.62±2.7%',
  'Pyrimidines',
 

In [6]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    print(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)
        

In [7]:
def compare(gt_list,table_obj):
    l1=table_obj.to_string().split()
    list1=[]
    for i in range(len(l1)):
        temp=l1[i].split("\\n")
        #l1.pop(i)
        for j in temp:
            list1.append(j)
    print(list1)
    if jaccard_similarity(gt_list,list1)>0.2:
        return True
    else:
        return False
            
       
    

In [8]:
def get_stats(dfDir,xmlDir):
    PDF_wise = {}
    list_dfDir  = os.listdir(dfDir)
    #print(list_dfDir)
    #list_xmlDir = os.listdir(xmlDir)
    for PDF in list_dfDir:
        Correctly_detected = 0
        PDF_wise[PDF] = {"total_Tables":0,"Camelot_detected":0,"Correctly_detected":0}
        xml_path = xmlDir +"/"+"%s.xml"%PDF
        #print("xml_path",xml_path)
        tables_in_PDF = os.listdir(dfDir+"/"+PDF)
        #print(tables_in_PDF)
        gt = get_labels(xml_path)
        #print("gt",gt.values())
        total_tables=len(gt.values())
        camelot_detected = len(tables_in_PDF)
        PDF_wise[PDF]["total_Tables"] = total_tables
        PDF_wise[PDF]["Camelot_detected"] = camelot_detected
        
        for table_gt in gt:
            pn_gt = table_gt.split("_")[-1]
            #print(pn_gt)
            for table_c in tables_in_PDF:
                pn_cam = table_c.split("_")[-1].split(".pkl")[0]
                #print(pn_cam)
                if pn_gt == pn_cam:
                    table_obj = pd.read_pickle(dfDir+"/"+PDF+"/"+table_c)
                    if compare(gt[table_gt],table_obj):
                        Correctly_detected += 1
        PDF_wise[PDF]["Correctly_detected"] = Correctly_detected
    return PDF_wise
                
        
        
        
    
    

In [9]:
dfDir = os.getcwd()+"/df"
xmlDir = os.getcwd()+"/xmls"
get_stats(dfDir,xmlDir)

['0', '1', '0', 'MCO', 'PL', '1', 'w', 'φ', 'v(w)', 'rv,w(φ)', 'ri', 'v(φ)', 'x', 'ξ', 'P', '(x)', 'Rx(ξ)', 'Ri(ξ)', '2']
['w', 'ξ', 'PL', 'P', 'φ', 'x', '(x)', 'v(w)', 'MCO']
['0', '1', '0', 'RSM', 'PC', '1', 'Fit', 'parametric', 'function', 'to', 'objective', 'function...', 'Fit', 'parametric', 'distribution', 'to', 'target', 'distrib...', '2', 'Heuristics', 'to', 'grow', 'trust', 'region', 'Cross-validation', 'for', 'regularization', '3', 'DOE', 'for', 'sample', 'points', 'Random', 'sampling', 'for', 'sample', 'points', '4', 'Axis', 'alignment', 'of', 'stencil', 'matters', 'Parametrization', 'can', 'address', 'axis', 'alignment', '5', 'Surrogate', 'minimization', 'not', 'always', 'easy', 'Implicit,', 'probabilistic', '‘minimization’', 'of', 'surr...', '6']
['DOE', 'easy', 'sampling', 'Axis', 'not', 'stencil', 'trust', 'region', 'alignment', 'Heuristics', 'points', 'to', 'of', 'Fit', 'minimization', 'for', 'Cross-validation', 'target', 'objective', 'always', '‘minimization’', 'Implic

{'0704.1274': {'total_Tables': 2,
  'Camelot_detected': 2,
  'Correctly_detected': 2},
 '0704.1411': {'total_Tables': 2,
  'Camelot_detected': 4,
  'Correctly_detected': 2},
 '0704.1198': {'total_Tables': 3,
  'Camelot_detected': 2,
  'Correctly_detected': 2},
 '0704.1269': {'total_Tables': 3,
  'Camelot_detected': 9,
  'Correctly_detected': 3},
 '0704.1028': {'total_Tables': 3,
  'Camelot_detected': 3,
  'Correctly_detected': 2}}

In [117]:
a = camelot.read_pdf("0704.1411.pdf",pages = "3")
a[0].df


,0,1,2,3,4
0,4,[5 2],1.005 ± .0018,[5 7],1.003 ± .0018
1,8,[13 04],1.097 ± .0016,[13 17],1.098 ± .0016
2,16,[23 04],1.162 ± .0014,[23 35],1.169 ± .0014
3,32,[45 10],1.232 ± .0013,[53 75],1.233 ± .0013
4,64,[103 024],1.282 ± .0012,[133 171],1.290 ± .0012
5,128,[235 126],1.332 ± .0010,–,–
6,256,[515 362],1.369 ± .0010,[561 753],1.375 ± .0010
7,1024,–,–,[2335 3661],1.428 ± .0008
